<a href="https://colab.research.google.com/github/ghostfm3/ksks/blob/master/SVD_R001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null
!echo mecab-config --dicdir"/mecab-ipadic-neologd"

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
mecab-config --dicdir/mecab-ipadic-neologd


In [3]:
pip install unidic-lite

     |████████████████████████████████| 47.3MB 93kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.7-cp36-none-any.whl size=47556593 sha256=40a92ff3849086a1e0e61c298c284d8c0ed85de6f9070b2c794f2f3777205bd2
  Stored in directory: /root/.cache/pip/wheels/a8/82/7d/086724645e33a575aafd0b1dae2835c37d2c00c6a0a96ee3a0
Successfully built unidic-lite


In [5]:
import numpy as np
import sys
import MeCab
sys.path.append('..')
def preprocess(text):
  text = text.lower()
  text = text.replace('.', ' .')
  words = text.split(' ')

  word_to_id = {}
  id_to_word = {}
  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id
      id_to_word[new_id] = word 
  
  corpus = np.array([word_to_id[w] for w in words])
  return corpus, word_to_id, id_to_word

def wakati(text):
    tagger = MeCab.Tagger('')
    tagger.parse('') 
    node = tagger.parseToNode(text)
    word_list = []
     
    while node:
        pos = node.feature.split(",")[0]
        #品詞を選択
        if pos in ["名詞","動詞","助詞","形容詞","代名詞","助動詞"]: 
            word = node.surface
            word_list.append(word)
        node = node.next
         
    return " ".join(word_list)

#m = MeCab.Tagger('-O wakati')
#sentence = m.parse('吾輩は猫である。名前はまだ無い。')
#text = sentence
text1 = 'PPIとはプロトンポンプ阻害薬のことです'
text = wakati(text1)
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)

print(id_to_word)

[0 1 2 3 4 5 6 7 8]
{0: 'ppi', 1: 'と', 2: 'は', 3: 'プロトン', 4: 'ポンプ', 5: '阻害', 6: 'の', 7: 'こと', 8: 'です'}


In [13]:
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
import MeCab

def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

def create_co_matrix(corpus, vocab_size, window_size=1):
    '''共起行列の作成
    :param corpus: コーパス（単語IDのリスト）
    :param vocab_size:語彙数
    :param window_size:ウィンドウサイズ（ウィンドウサイズが1のときは、単語の左右1単語がコンテキスト）
    :return: 共起行列
    '''
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

def ppmi(C, verbose=False, eps = 1e-8):
    '''PPMI（正の相互情報量）の作成
    :param C: 共起行列
    :param verbose: 進行状況を出力するかどうか
    :return:
    '''
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)
            M[i, j] = max(0, pmi)

            if verbose:
                cnt += 1
                if cnt % (total//100 + 1) == 0:
                    print('%.1f%% done' % (100*cnt/total))
    return M

def wakati(text):
    tagger = MeCab.Tagger('')
    tagger.parse('') 
    node = tagger.parseToNode(text)
    word_list = []
     
    while node:
        pos = node.feature.split(",")[0]
        #品詞を選択
        if pos in ["名詞","動詞","助詞","副詞","形容詞","代名詞","助動詞"]: 
            word = node.surface
            word_list.append(word)
        node = node.next
         
    return " ".join(word_list)



text1 = 'PPIとはプロトンポンプ阻害薬のことです'
text = wakati(text1)
#text='You say goodbye and I say hello'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(id_to_word)
C = create_co_matrix(corpus, vocab_size, window_size=1)
W = ppmi(C)

U, S, V = np.linalg.svd(W)

In [14]:
print(C[0])

[0 1 0 0 0 0 0 0 0]


In [15]:
print(W[0])

[0. 3. 0. 0. 0. 0. 0. 0. 0.]


In [22]:
print(id_to_word[0],U[0, :5])
print(id_to_word[1],U[1, :5])
print(id_to_word[2],U[2, :5])
print(id_to_word[3],U[3, :5])
print(id_to_word[4],U[4, :5])
print(id_to_word[5],U[5, :5])
print(id_to_word[6],U[6, :5])
print(id_to_word[7],U[7, :5])
print(id_to_word[8],U[8, :5])

ppi [ 0.          0.39113787 -0.52034736  0.         -0.48241627]
と [-0.53010255  0.          0.          0.66094804  0.        ]
は [ 0.          0.49095163 -0.4787887   0.          0.0427042 ]
プロトン [-0.46796507  0.          0.          0.251292    0.        ]
ポンプ [0.0000000e+00 4.6038607e-01 1.1574242e-17 0.0000000e+00 7.2863007e-01]
阻害 [-0.46796507  0.          0.         -0.251292    0.        ]
の [0.         0.49095163 0.4787887  0.         0.0427042 ]
こと [-0.53010255  0.          0.         -0.66094804  0.        ]
です [ 0.          0.39113787  0.52034736  0.         -0.48241627]
